In [12]:
import string

def remove_chars_from_text(text, chars):
    # return "".join([ch for ch in text if ch not in chars])
    content = ''
    for ch in text:
        if ch not in chars:
            content = content + ''.join(ch)
        else:
            content = content + ''.join(' ')
    return content

import re

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)
spec_chars = string.punctuation + '\r' + '\n\xa0«»\t—…' 


In [13]:
def predict_contry (Text_post):
    import pickle
    from sklearn.linear_model import LogisticRegression
    from sklearn.feature_extraction.text import CountVectorizer

    
    with open('contry_post-model.bin', 'rb') as f_out:
        log_clf, X_train_words=pickle.load( f_out)
    x_temp = []
    content = str(Text_post).lower()  
    content = remove_emojis(content)
    content = remove_chars_from_text(content, spec_chars)
    content = remove_chars_from_text(content, string.digits)
    for i in range(1, 10):
        content = content.replace('  ', ' ')
    x_temp.append(content)
       
    
    count = CountVectorizer(vocabulary = set(X_train_words))
    X_t = count.transform(x_temp)
    accuracy = log_clf.predict_proba(X_t)
    ix = accuracy.argmax(1).item()
    # print(f'predicted class = {log_clf.classes_[ix]} and confidence = {accuracy[0,ix]:.2%}')
    predicted_class =log_clf.classes_[ix]
    confidence = accuracy[0,ix]    
    return predicted_class, confidence

In [14]:


def get_telegram_adv(user_name, Dir_Path= 'Set_A', iter_number=25, text_limit=True):
    '''
    :param user_name - Название телеграм канала:
    :param iter_number - Количество иттераций по дате публикации постов каждый шаг 20 постов max:
    :return:
    :text (str) - Текст постов канала (очищенный)
    web_names_set - список ссылок на внешние ресурсы, на которые ссылается канал
    telegram_names_set - список телеграм каналов на которые ссылантся канал
    set(lang) (set) - язык постов
    av_post_len - среднее значение символов в одном посте
    '''


    import requests
    from fake_useragent import UserAgent
    import random
    import time
    from bs4 import BeautifulSoup
    import re
    import os
    import io
    import json



    url = 'https://t.me/s/{}'.format(user_name)
    useragent = {'User-Agent': UserAgent().random}
    text = ''

    try:
        r = requests.get(url, headers=useragent)
        time.sleep(random.randrange(1, 10))
        text = get_text(r, user_name, Dir_Path)
        number = re.findall(r"data-post=\"{}\/(\d+)".format(user_name), r.text)
        # print(number)
        if not len(number) == 0:
            last_post_num = int(number[-1])
            post_num = last_post_num
            if last_post_num < (iter_number+1)*20:
                iter_number = last_post_num//20 - 1
            for items in range(iter_number):
                r = requests.get('https://t.me/s/{}?before={}'.format(user_name, post_num), headers=useragent)
                get_text(r,user_name + str(post_num), Dir_Path)
                time.sleep(random.randrange(1, 7))
                temp_posts = re.findall(r"data-post=\"{}\/(\d+)".format(user_name), r.text)
                post_num = int(temp_posts[0])-1

    except Exception as ex:
        print(ex)
        return

def get_text(r, filename, Dir_Path):
    # Извлечение текста
    from bs4 import BeautifulSoup
    import re
    text = ''
    text_post = ''
    soup = BeautifulSoup(r.text, 'lxml')
    popular = soup.find_all('div', {'class': 'tgme_widget_message_text'})
    iterr = 0
    for item in popular:
        soup_temp = BeautifulSoup(item.text, 'lxml')
        text_post = re.sub(r'https?:\/\/.*[\r\n]*', '', soup_temp.text, flags=re.MULTILINE)
        text = text + '\n'+ text_post
        predicted_class, confidence = predict_contry(text_post)
        print('RES -{}, {}, {}, {}%'.format((predicted_class==Dir_Path) and (confidence>0.9), predicted_class, Dir_Path, confidence))
        if (len(text_post) > 150) and (len(text_post) < 2000) and (lang_detect(text_post) == 'ru') and (predicted_class==Dir_Path) and (confidence>0.75):
            try:
                with open('./telegram_Dataset/{}/@{}_telegram.txt'.format(Dir_Path, filename+'_'+str(iterr)), "w", encoding="utf-8") as some_file:
                    print(text_post, file=some_file)
                    some_file.close()
                    iterr = iterr+1
            except Exception as ex:
                print(ex)
    return text

def lang_detect (text = ''):
    # Определение языка канала
    from bs4 import BeautifulSoup
    from langdetect import detect
    import re

    if text !='':
        try:
            lang = detect(text[400:])
            print('Язык поста {}'.format(lang))
            return lang
        except Exception as ex:
            print(ex)
            return 'no text'
    else:
        return 'no text'

def pipline(content):
    import string
    import re
    from tokenizer_exceptions import normalizer_exc_rus
    print(content[:200])
    content = normalizer_exc_rus(content).lower()
    spec_chars = string.punctuation + '\xa0«»\t—…'
    content = re.sub('\n', ' ', content)
    content = re.sub('\r', ' ', content)
    content = remove_chars_from_text(content, spec_chars)
    content = remove_chars_from_text(content, string.digits)
    content = " ".join(content.split())
    print(content[:200])
    return content

def remove_chars_from_text(text, chars):
    # return "".join([ch for ch in text if ch not in chars])
    content = ''
    for ch in text:
        if ch not in chars:
            content = content + ''.join(ch)
        else:
            content = content + ''.join(' ')
    return content




def main():
    import os


    next_iter_names = {
        # 'china': ['maslovasia', 'china80s', 'raspp_info', 'prchand', 'awaken_dragon', 'asiatica_ru'],
        # 'africa': ['zangaro', 'africablack', 'natasakado_official', 'dnobangui', 'africafordummies', 'westernafrica', 'meskob', 'catcherinsudan', 'africanists'],
        # 'middleeast': ['syriaagency', 'turk_gambit_ca', 'shatergaddafi', 'turkkulubu', 'mideastr', 'tangermanar', 'marocrus', 'arabstatesofgulf', 'assadstash', 'meastru', 'strana_tuaregov']
        # 'latamerica': ['pqntc1', 'privetfidel', 'altimurla','favelasemrus', 'pincheponchito', 'laprimaveradelpatriarca', 'lat_america', 'tupireport', 'venezuelanewsnetwork', 'sputnikmundo']
        # 'news_ru': ['boris_rozhin', 'rt_russian', 'infantmilitario', 'svarschiki', 'divgen'],
        # 'news_ua': ['rezident_ua', 'taynaya_kantselyariya', 'the_military_analytics', 'spletnicca'],
        #'IranPakistanAfganistan': ['IranPakistanAfganistan', 'afgbezparandzhi'],
        #'KNDR': ['RusEmbDPRK'],
        #'Iran': ['irandezhurniy', 'paxIranica', 'IranPakistanAfganistan'],
        #'Afganistan': ['afgbezparandzhi', 'IranPakistanAfganistan'],
        #'Ethiopia': ['Ethiopia_tezeta', 'meskob'],
        #'Pakistan': ['IranPakistanAfganistan'],
        #'China': ['china80s', 'raspp_info', 'prchand'],
        #'India': ['india_tv2020', 'ninerasas', 'indiareads', 'speciallassi', 'indiaanalytics', 'Indosphere', 'India_sangrahalaya'],
        #'Turkey': ['turkeyabout', 'turkkulubu'],
        #'Libya': ['ShaterGaddafi'],
        #'Morocco': ['tangermanar', 'marocrus'],
        #'Syria': ['NovostiDamask'],
        #'CAR': ['dnobangui'],# Центрально-африканская Республика
        #'Sudan': ['CatcherInSudan'],
        #'Argentina': ['argentinarusa'],
        'Iran': ['IranPakistanAfganistan'],
        'Afganistan': [ 'IranPakistanAfganistan'],
        'Pakistan': ['IranPakistanAfganistan'],
    }

    
    for item in next_iter_names.keys():

        if not os.path.exists('./telegram_Dataset/{}'.format(item)):
            os.makedirs('./telegram_Dataset/{}'.format(item))


        for items in next_iter_names[item]:
            # print('Канал {}'.format(items))
            get_telegram_adv(items, item)


In [15]:
main()

RES -False, India, Iran, 0.2229983928636596%
Язык поста ru
RES -False, India, Iran, 0.27630381154858513%
No features in text.
RES -False, India, Iran, 0.3284334586838847%
Язык поста ru
RES -False, India, Iran, 0.9692221283676427%
Язык поста ru
RES -False, India, Iran, 0.9692221283676427%
Язык поста ru
RES -False, Iran, Iran, 0.29553720415452855%
Язык поста ru
RES -False, India, Iran, 0.40018459691638447%
Язык поста ru
RES -False, Iran, Iran, 0.21285754661645406%
No features in text.
RES -False, CAR, Iran, 0.19315591035459864%
No features in text.
RES -False, Iran, Iran, 0.18445395630097175%
No features in text.
RES -False, Afganistan, Iran, 0.22097694326492576%
No features in text.
RES -False, Iran, Iran, 0.6134734085637487%
Язык поста ru
RES -False, Iran, Iran, 0.38249074930040305%
No features in text.
RES -False, CAR, Iran, 0.19143067240391787%
Язык поста ru
RES -False, India, Iran, 0.5581491931187765%
Язык поста ru
RES -False, Iran, Iran, 0.32687300136745034%
No features in text.
RE

C:\Users\user\AppData\Local\Temp\ipykernel_12200\921660705.py:62: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup_temp = BeautifulSoup(item.text, 'lxml')


RES -False, India, Iran, 0.1858331130716138%
No features in text.
RES -False, India, Iran, 0.2229983928636596%
Язык поста ru
RES -False, India, Iran, 0.27630381154858513%
No features in text.
RES -False, India, Iran, 0.3284334586838847%
Язык поста ru
RES -False, India, Iran, 0.9692221283676427%
Язык поста ru
RES -False, India, Iran, 0.9692221283676427%
Язык поста ru
RES -False, Iran, Iran, 0.29553720415452855%
Язык поста ru
RES -False, India, Iran, 0.40018459691638447%
Язык поста ru
RES -False, Iran, Iran, 0.21285754661645406%
No features in text.
RES -False, CAR, Iran, 0.19315591035459864%
No features in text.
RES -False, Iran, Iran, 0.18445395630097175%
No features in text.
RES -False, Afganistan, Iran, 0.22097694326492576%
No features in text.
RES -False, Iran, Iran, 0.6134734085637487%
Язык поста ru
RES -False, Iran, Iran, 0.38249074930040305%
No features in text.
RES -False, CAR, Iran, 0.19143067240391787%
Язык поста ru
RES -False, India, Iran, 0.5581491931187765%
Язык поста ru
RE

C:\Users\user\AppData\Local\Temp\ipykernel_12200\921660705.py:62: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup_temp = BeautifulSoup(item.text, 'lxml')


RES -False, Iran, Iran, 0.5818897857281556%
Язык поста ru
RES -False, India, Iran, 0.8091585568619818%
RES -False, CAR, Iran, 0.2483638956006645%
No features in text.
RES -False, India, Iran, 0.5256026003460174%
Язык поста ru
RES -False, Afganistan, Iran, 0.41675631343757147%
Язык поста ru
RES -False, Afganistan, Iran, 0.983143926498918%
RES -False, India, Iran, 0.26248048199496776%
Язык поста ru
RES -False, Iran, Iran, 0.2437688771548768%
No features in text.
RES -False, Iran, Iran, 0.2716641131595329%
No features in text.
RES -False, China, Iran, 0.4000919378327801%
Язык поста ru
RES -False, Iran, Iran, 0.2588041948144889%
No features in text.
RES -False, CAR, Iran, 0.23735648844696713%
RES -False, India, Iran, 0.21233058661875595%
No features in text.
RES -False, Iran, Iran, 0.3305080327982812%
No features in text.
RES -False, Iran, Iran, 0.5248805340898774%
Язык поста ru
RES -False, Iran, Iran, 0.3898550629333023%
Язык поста ru
RES -False, India, Iran, 0.20205447561144999%
No featu